Seunghwan NN

In [2]:
import visualkeras
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import pickle
from torchvision import models
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import multiprocessing
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import tensorflow.keras.applications as LayerApp
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.applications import ResNet50
import os
import time
import pickle
import numpy as np
import absl.logging

In [11]:
class CustomCNN(Model):
    def __init__(self, num_classes=2):
        super(CustomCNN, self).__init__()
        # Add L2 regularization to the convolutional layers
        #self.conv1 = Conv1D(32, 3, activation='relu', padding='same', kernel_regularizer=l2(0.00001))
        self.conv1 = Conv1D(32, 3, activation='relu', padding='same')
        #self.bn1 = BatchNormalization()
        self.pool1 = MaxPooling1D(2)
        #self.dropout1 = Dropout(0.2)

        #self.conv2 = Conv1D(64, 3, activation='relu', padding='same', kernel_regularizer=l2(0.00001))
        self.conv2 = Conv1D(64, 3, activation='relu', padding='same')
        #self.bn2 = BatchNormalization()
        #self.pool2 = MaxPooling1D(2)
        #self.dropout2 = Dropout(0.2)

        self.flatten = Flatten()
        self.dense1 = Dense(256, activation='relu')
        self.dropout3 = Dropout(0.2)
        self.dense2 = Dense(256, activation='relu')
        self.dense3 = Dense(num_classes, activation='softmax')

    def call(self, inputs, training=False):
        # 시퀀스 길이 대신 특징 차원에 대한 차원 추가
        inputs = tf.expand_dims(inputs, axis=-1)
        x = self.conv1(inputs)
        #x = self.bn1(x, training=training)
        x = self.pool1(x)
        #x = self.dropout1(x, training=training)

        x = self.conv2(x)
        #x = self.bn2(x, training=training)
        #x = self.pool2(x)
        #x = self.dropout2(x, training=training)

        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dropout3(x, training=training)
        x = self.dense2(x)
        return self.dense3(x)

class FNN(Model):
    def __init__(self, num_classes=2):
        super(FNN, self).__init__()
        self.dense1 = Dense(256, activation='relu')
        self.dense2 = Dense(256, activation='relu')
        self.dense3 = Dense(256, activation='relu')
        self.dense_last = Dense(num_classes, activation='softmax')

    def call(self, inputs, training=False):
        # 시퀀스 길이 대신 특징 차원에 대한 차원 추가
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        return self.dense_last(x)


def model_fnn(input_witdh):
    model = FNN()
    model.build((None, input_witdh))
    return model

def model_custom(input_width):
    model = CustomCNN()
    model.build((None, input_width))
    return model

def model_resnet(input_width,layer_depth):
    # 사전 훈련된 EfficientNetB2 모델 로드
    base_model = ResNet50(include_top=False, weights="imagenet", input_shape=(input_width, input_width, 3))

    total_layers = len(base_model.layers)
    trainable_layers = int(total_layers * layer_depth)  # 하위 j만 훈련

    for layer in base_model.layers[:-trainable_layers]:
        layer.trainable = False

    # 커스텀 층 추가
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)  # 배치 정규화 추가
    x = Dropout(0.2)(x)  # 드롭아웃 추가

    predictions = Dense(2, activation='softmax')(x)  # 다중 클래스 분류를 위한 설정

    # 새로운 모델 정의
    model = Model(inputs=base_model.input, outputs=predictions)
    return model


def model_efficientnet(input_width,layer_depth):
    # 사전 훈련된 EfficientNetB2 모델 로드
    base_model = LayerApp.EfficientNetV2M(include_top=False, weights="imagenet", input_shape=(input_width, input_width, 3))

    total_layers = len(base_model.layers)
    trainable_layers = int(total_layers * layer_depth)  # 전체가 1일 때, layer_depth만큼 ratio에 대해 학습

    for layer in base_model.layers[:-trainable_layers]:
        layer.trainable = False

    # 커스텀 층 추가
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)  # 배치 정규화 추가
    x = Dropout(0.4)(x)  # 드롭아웃 추가

    predictions = Dense(2, activation='softmax')(x)  # 다중 클래스 분류를 위한 설정

    # 새로운 모델 정의
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

model1 = models.Sequential()
model1.add(tf.keras.Input(shape=(16,)))
model1.add(model_fnn(16))
# model1.add(layers.Dense(256, activation='relu'))
# model1.add(layers.Dense(256, activation='relu'))
# model1.add(layers.Dense(256, activation='relu'))
# model1.add(layers.Dense(2, activation='softmax'))
# model1.summary()
visualkeras.layered_view(model1)

ValueError: Could not save to JPEG for display

Sewoon NN

In [7]:
class ConvNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ConvNetClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(
            64 * 8 * 8, 512
        )  # 이미지 크기가 32x32이므로 최종 feature map 크기는 8x8입니다.
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)  # Flatten the feature maps
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model2 = models.Sequential()
model2.add(layers.conv2d)

visualkeras.layered_view(model2)

AttributeError: 'ConvNetClassifier' object has no attribute 'layers'

In [12]:
from tensorflow.keras.applications import ResNet50
from keras.layers import  Dropout
from tensorflow.keras.optimizers import Adam
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model

model = ResNet50(
      input_shape = (224,224,3),
      include_top = False,
      weights = 'imagenet'
    )

for layers in model.layers:
    layers.trainable = False

x = Flatten()(model.output)
x = Dropout(0.5)(x)
x = Dense(1, activation = "sigmoid")(x)

model = Model(model.input, x)
model.compile(loss = "binary_crossentropy", optimizer = "RMSprop", metrics = ["accuracy"])
model.summary()


# 커스텀 층 추가
x = model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)  # 배치 정규화 추가
x = Dropout(0.2)(x)  # 드롭아웃 추가

predictions = Dense(2, activation='softmax')(x)  # 다중 클래스 분류를 위한 설정

# 새로운 모델 정의
model = Model(inputs=model.input, outputs=predictions)


94765736/94765736 [==============================] - 9s 0us/step

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_8[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                       

ValueError: Input 0 of layer "global_average_pooling2d" is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: (None, 1)

In [14]:
visualkeras.layered_view(model).show()